# An introduction to simulations with msprime

The goal of this notebook is to introduce msprime but linking it to the lectures and in particular the Wright-Fisher model. We will perform the same build-up as we did with the WF. 

Dependencies: plotting

1. Ancestry simulations - making genealogies
2. Adding mutations
3. Plotting trees

In [ ]:
from IPython.display import SVG, display
import tskit
import msprime
import tqdm

# The two lines below required for JupyterLite
import micropip
await micropip.install('jupyterquiz')

# Add local code here
#import genealogical_analysis_workshop

#workbook = genealogical_analysis_workshop.setup_workbook1()
#display(workbook.setup)

In [ ]:
ts = msprime.sim_ancestry(3)
ts

In [ ]:
SVG(ts.draw_svg(size=(600, 200)))